<span>Note: write.save and spark.read.load functions work only on spark version 3.0.0</span>
<code style="font-size:10px; background:none">
train.write.save("train.parquet")
spark.read.load("train.parquet")
</code>
<span>Read parquet file with pyarrow.parquet and convert into pandas and then convert into spark dataframe</span>
<code style="font-size:10px; background:none">
import pyarrow.parquet as pq
train.write.parquet("train.parquet")
train = pq.read_table('train.parquet')
spark_session = pyspark.sql.SparkSession.builder.getOrCreate()
train_pandas = train.to_pandas()
train_spark = spark_session.createDataFrame(train_pandas)
</code>

In [4]:
import pyspark as spark

# Import Spark NLP  
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

from sparknlp.pretrained import PretrainedPipeline
import databricks.koalas as ks

from pyspark.sql.functions import col
import pyarrow.parquet as pq
import pandas as pd

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.__version__)

Spark NLP version 2.4.5
Apache Spark version: 2.4.6


Loading the spark 2.4.6 dataframe is not figured out right now

In [3]:
import sparknlp
import pyarrow.parquet as pq
import pandas as pd

In [4]:
spark = sparknlp.start()
df_spark = spark.read.csv('data/amazon_reviews_us_Electronics_v1_00.tsv', header='true', inferSchema='true', sep='\t')
# df_spark.write.parquet("data/df.parquet") # Error if the file already exists.
df_pq = pq.read_table('data/df.parquet')
df_pandas = df_pq.to_pandas()

In [7]:
counts = df_pandas.groupby('product_id').count()
products_10_or_more_reviews = counts.index[counts.iloc[:,0] >= 10]
mask = df_pandas.product_id.map(lambda x: x in products_10_or_more_reviews).values
train_for_sentiment = df_pandas.loc[~mask,:]
df_pandas = df_pandas.loc[mask, :]

In [8]:
len(train_for_sentiment), len(df_pandas), len(train_for_sentiment) + len(df_pandas)

(346460, 2747409, 3093869)

In [9]:
document = DocumentAssembler()\
    .setInputCol("review_body")\
    .setOutputCol("document")

In [ ]:
# downloading pretrained embeddings
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")